In [1]:
def per_channel_quant(tensor,dim,dtype):
    pass

In [24]:
import torch
z = torch.randint(low=-150,high=150,size=(3,3)) + torch.randn((3,3))

In [51]:
def quantize_tensor(tensor,dtype=torch.int8):
    r_max = tensor.max()
    r_min = tensor.min()
    tensor_dtype = tensor.dtype
    q_max = torch.iinfo(dtype).max
    q_min = torch.iinfo(dtype).min
    #scale must be in higher precision
    scale = torch.tensor((r_max-r_min)/(q_max-q_min),dtype=tensor_dtype)
    zero = int(torch.round(q_min - (r_min/scale)))
    if zero > q_max:
        zero = q_max
    if zero < q_min:
        zero = q_min
    q_tensor = tensor/scale + zero
    rounded_tensor = torch.round(q_tensor)
    rounded_tensor = rounded_tensor.clamp(q_min,q_max).to(dtype)
    return rounded_tensor,scale,zero

In [44]:
#define output dim for axis 0or1
dim=1
q_max=127
output_dim = z.shape[dim]
#create 
qs,scales,zeros = [],[],[]
#iter over output dim
for index in range(output_dim):
    sub_tensor = z.select(dim,index)
    q_sub,scale,zero = quantize_tensor(sub_tensor,torch.int8)
    qs.append(q_sub)
    scales.append(scale)
    zeros.append(zero)
qs = torch.stack(qs,dim)
qs
de_qs = []
for index in range(output_dim):
    sub_tensor = qs.select(dim,index)
    sub_scale = scales[index]
    sub_zero = zeros[index]
    de_qs.append(de_quantize(sub_tensor,sub_scale,sub_zero,torch.float32))
de_qs = torch.stack(de_qs,dim)

q  : 
tensor([-128,  -99,  127], dtype=torch.int8)
zero : -6
scale : 1.1037447452545166
without upcasting q-z : 
tensor([-122,  -93, -123], dtype=torch.int8)
with upcasting q-z : 
tensor([-122.,  -93.,  133.])
q  : 
tensor([ 127, -128,  -23], dtype=torch.int8)
zero : 122
scale : 0.5962054133415222
without upcasting q-z : 
tensor([  5,   6, 111], dtype=torch.int8)
with upcasting q-z : 
tensor([   5., -250., -145.])
q  : 
tensor([  27, -128,  127], dtype=torch.int8)
zero : -92
scale : 0.5932983756065369
without upcasting q-z : 
tensor([119, -36, -37], dtype=torch.int8)
with upcasting q-z : 
tensor([119., -36., 219.])


/var/folders/xw/knd54pd91c7cw57n21dntvx80000gn/T/ipykernel_24228/2609975628.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  scale = torch.tensor((r_max-r_min)/(q_max-q_min),dtype=tensor_dtype)


In [14]:
#select takes dim and index
z.select(1,2)

tensor([116.5177, 101.5764, -78.5647])

In [9]:
z

tensor([[111.3387, -15.4664, 116.5177],
        [-12.3060,  26.3288, 101.5764],
        [105.9068, -57.8360, -78.5647]])

In [54]:
def de_quantize(tensor,scale,zero,dtype=torch.float32):
    deq_tensor = scale * (tensor.float() - zero)
    return deq_tensor

In [61]:
def per_channel_quant(tensor,dim,dtype=torch.int8):
    output_dim = tensor.shape[dim]
    qs,scales,zeros = [],[],[]
    for index in range(output_dim):
        sub_tensor = tensor.select(dim,index)
        q,s,z = quantize_tensor(sub_tensor,dtype)
        qs.append(q)
        scales.append(s)
        zeros.append(z)
    qs = torch.stack(qs,dim)
    return qs,scales,zeros

In [89]:
def per_channel_de_quant(tensor,scales,zeros,dim,dtype=torch.float32):
    output_dim = tensor.shape[dim]
    de_qs = []
    for index in range(output_dim):
        sub_tensor = tensor.select(dim,index)
        de_q = de_quantize(sub_tensor,scales[index],zeros[index],dtype)
        de_qs.append(de_q)
    de_qs = torch.stack(de_qs,dim)
    return de_qs

In [90]:
import torch
z = torch.randint(low=-150,high=150,size=(3,3)) + torch.randn((3,3))

### dim =0

In [91]:
z

tensor([[116.2372, -16.7232, -35.4347],
        [ 36.0951, -82.5614, 103.2559],
        [  4.6823, -67.4668,  11.3159]])

In [92]:
dim_0 = 0
q_z_0,scales_0,zeros_0 = per_channel_quant(z,dim_0,dtype=torch.int8)

/var/folders/xw/knd54pd91c7cw57n21dntvx80000gn/T/ipykernel_24228/2609975628.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  scale = torch.tensor((r_max-r_min)/(q_max-q_min),dtype=tensor_dtype)


In [93]:
q_z_0

tensor([[ 127,  -96, -128],
        [  35, -128,  127],
        [ 105, -128,  127]], dtype=torch.int8)

In [94]:
de_q_z_0 = per_channel_de_quant(q_z_0,scales_0,zeros_0,dim_0)

In [95]:
de_q_z_0

tensor([[115.9844, -16.6542, -35.6875],
        [ 36.4348, -82.3426, 103.4747],
        [  4.6343, -67.3514,  11.4312]])

In [96]:
z

tensor([[116.2372, -16.7232, -35.4347],
        [ 36.0951, -82.5614, 103.2559],
        [  4.6823, -67.4668,  11.3159]])

In [ ]:
import seaborn as sns
sns.heatmap(abs(z-de_q_z_0),annot=True,titl)